In [64]:
import json

with open('export/users.json', 'r') as file:
    users = json.loads(file.read())
    users = [(user['id'], user['name']) for user in users]
    users = dict(users)

print(f'{len(users)} users found')

824 users found


In [98]:
from glob import glob
import datetime
import re

all_messages = []

def replace_user_id(match):
    id = match.group(1)
    return users.get(id, 'unknown_user')

ignored_channels = []
ignored_message_subtypes = [
    'channel_join', 
    'channel_leave', 
    'channel_purpose', 
    'file_share', 
    'channel_archive', 
    'bot_message'
]

channel_dirs = glob('export/*/')
for channel_dir in channel_dirs:
    channel_name = channel_dir.split('/')[-2]
    if channel_name in ignored_channels:
        continue
    days = os.listdir(channel_dir)
#     print(f'{len(days)} days in {channel_name}')
    for day in days:
#         print(f'Processing {channel_name}/{day}')
        with open(f'{channel_dir}{day}', 'r') as file:
            day = json.loads(file.read())
            for message in day:
                if message.get('subtype') in ignored_message_subtypes:
                    continue
                user = users.get(message.get('user'))
                    
                msg = message.get('text')
                if msg is None:
                    msg = ''

                text = re.sub(r'<@(.+)>', replace_user_id, msg)
                timestamp = datetime.datetime.fromtimestamp(int(float(message.get('ts'))))

                all_messages.append({'channel': channel_name, 'timestamp': timestamp, 'user': user, 'text': text})

print(f'{len(all_messages)} messages found')
# for msg in all_messages[:2]:
#     print(f"{msg['channel']} {msg['timestamp']} <{msg['user']}> {msg['text']}")

218871 messages found


In [103]:
user_messages = {}

for user_id, username in users.items():
    messages = [msg for msg in all_messages if msg['user'] == username]
    user_messages[username] = messages

In [115]:
import markovify

text = '\n'.join([msg['text'] for msg in user_messages['kilian']])
text_model = markovify.Text(text)

for i in range(10):
    print(text_model.make_short_sentence(140))

Hatte da mal zu eng gedacht. ich kann den Cursor nur in meiner inbox landet.
Verboten wird sowas von großartig Ich hab's geliebt!
Was man wrappt und wie reagierst du sonst wahrscheinlich eher ueberhaupt kein geld bekommen :smile: sehr traurig^^ Weil man's nicht braucht?
Meine Safari Reading List ist viel mehr daran, die FB Apps auf jeden Fall bedeutend mehr, das stimmt.
Natürlich nur mit großen Waffen, bin mir da ziemlich sicher, weil die den Service selbst nutzt?
Aber ich würde mir was performanteres als Foundation für alle Zeit in Schach halten und Beine abhacken, dann geht's recht zügig.
Gesehen hab ich dann erstmal nachrennen muss.
Will nicht unnötig darauf rumsitzen wenn ein Auftraggeber da gern was hätte?
:blush: Ich find's ziemlich schade, dass die Adresse dir gehört.
Uuuh stark, thx für die nahtlose Usability führt vermutlich nicht reichen. du musst deine Geräte kurz abgeben.
